<a href="https://colab.research.google.com/github/m-wessler/nbm-verification/blob/main/NBM_Text_Product_Parser_MWE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd

from tqdm import tqdm
from datetime import datetime, timedelta

In [4]:
# Fixed forecast length (15 periods, NBE)
forecast_periods = {'NBE':15,}
fixed_columns = {'NBE':59,}

filename = 'blend_nbetx.t01z.txt'

with open(filename, 'r') as rfp:
    file_data = np.array(rfp.readlines())

In [5]:
agg_df = pd.DataFrame()
init_datetime = None

starts = np.where(np.char.find(file_data, "NBE") != -1)[0]
ends = np.append((starts-1)[1:], len(file_data)-1)

for start, end in tqdm(zip(starts, ends),
                       total=len(starts), desc=f'Parsing {filename}'):

    site_block = file_data[start:end]

    # ONLY PROCESS DATETIME ON FIRST ENTRY, consistent throughout file
    if init_datetime == None:
        site_id, prod_id, init_date, init_hour, timezone = [
            site_block[0].split(' ')[i] for i in [1, 5, 10, 12, 13]]

        init_datetime = datetime.strptime(
            f'{init_date} {init_hour}', '%m/%d/%Y %H%M')

        # First column is ALWAYS the next day (NBE)
        fhr0_datetime = (init_datetime + timedelta(days=1)).replace(
            hour=int(site_block[2][8:10]))

    else:
        site_id = site_block[0].split(' ')[1]

    datetime_index = [fhr0_datetime + timedelta(hours=12*i)
        for i in range(0, forecast_periods[prod_id])]

    site_matrix = {'datetime':datetime_index,
                   'site_id':[site_id]*forecast_periods[prod_id]}

    for line in site_block[3:]:
        var_id = line[:4]
        var_raw = line[7:67].rstrip()
        var_data = [var_raw[i:i+3] for i in range(0, fixed_columns[prod_id], 4)]
        site_matrix[var_id] = var_data

    agg_df = pd.concat([agg_df,
        pd.DataFrame(site_matrix).set_index(['datetime', 'site_id'])])

agg_df.sort_index(inplace=True)
agg_df

Parsing blend_nbetx.t01z.txt: 100%|██████████| 9589/9589 [18:24<00:00,  8.68it/s]


FHR  TXN  XND  TMP  TSD  DPT  DSD  SKY  SSD  WDR  ...  \
datetime   site_id                                                    ...   
2024-05-16 086092    23   90    3   82    4   73    3   69   22   23  ...   
           188557    23   65    2   63    2   60    2   78   27    3  ...   
           220792    23   85    2   82    2   68    2   11   25   31  ...   
           2A1       23   71    1   65    2   61    2   58   35   30  ...   
           2M2       23   76    1   71    2   63    2   26   22   31  ...   
...                 ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
2024-05-23 ZNPU1    191   87    2   83    4   24    4   24   17   24  ...   
           ZNRN5    191   77    1   74    4   12    4   12    9   24  ...   
           ZONM8    191   58    6   56    5   36    3   64   22   26  ...   
           ZSFO1    191   82    4   76    4   62    4   42   24   25  ...   
           ZVLT2    191   94    3   88    3   72    2   25   26   17  ...   

                    PZR  PSN  PPL  PRA  S12  SLV  I12  S24  SOL  SWH  
datetime   site_id                                                    
2024-05-16 086092     0    0    0   21    0  120    0        54    2  
           188557     0    0    0    4    0  100    0        12    1  
           220792     0    0    0    0    0   95    0        68    0  
           2A1        0    0    0   29    0  110    0        30  NaN  
           2M2        0    0    0    1    0  100    0        44  NaN  
...                 ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  
2024-05-23 ZNPU1      0    0    0    0    0   94    0        64  NaN  
           ZNRN5      0    0    0    0    0  100    0        67  NaN  
           ZONM8      0    0    0   53    0   69    0         5  NaN  
           ZSFO1      0    0    0   52    0  100    0        13  NaN  
           ZVLT2      0    0    0    7    0  120    0        66  NaN  

[143835 rows x 29 columns]

In [10]:
output_csv_file = f'blend_nbetx.{datetime.strftime(init_datetime, "%Y%m%d")}.t13z.csv'
agg_df.to_csv(output_csv_file)

In [12]:
pd.read_csv(output_csv_file).set_index(['datetime', 'site_id'])

<ipython-input-12-3c0307da9d4d>:1: DtypeWarning: Columns (18,28) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv(output_csv_file).set_index(['datetime', 'site_id'])


FHR   TXN   XND   TMP   TSD   DPT   DSD   SKY  \
datetime            site_id                                                   
2024-05-16 00:00:00 086092     23  90.0   3.0  82.0   4.0  73.0   3.0  69.0   
                    188557     23  65.0   2.0  63.0   2.0  60.0   2.0  78.0   
                    220792     23  85.0   2.0  82.0   2.0  68.0   2.0  11.0   
                    2A1        23  71.0   1.0  65.0   2.0  61.0   2.0  58.0   
                    2M2        23  76.0   1.0  71.0   2.0  63.0   2.0  26.0   
...                           ...   ...   ...   ...   ...   ...   ...   ...   
2024-05-23 00:00:00 ZNPU1     191  87.0   2.0  83.0   4.0  24.0   4.0  24.0   
                    ZNRN5     191  77.0   1.0  74.0   4.0  12.0   4.0  12.0   
                    ZONM8     191  58.0   6.0  56.0   5.0  36.0   3.0  64.0   
                    ZSFO1     191  82.0   4.0  76.0   4.0  62.0   4.0  42.0   
                    ZVLT2     191  94.0   3.0  88.0   3.0  72.0   2.0  25.0   

                              SSD   WDR  ...   PZR   PSN   PPL   PRA   S12  \
datetime            site_id              ...                                 
2024-05-16 00:00:00 086092   22.0  23.0  ...   0.0   0.0   0.0  21.0   0.0   
                    188557   27.0   3.0  ...   0.0   0.0   0.0   4.0   0.0   
                    220792   25.0  31.0  ...   0.0   0.0   0.0   0.0   0.0   
                    2A1      35.0  30.0  ...   0.0   0.0   0.0  29.0   0.0   
                    2M2      22.0  31.0  ...   0.0   0.0   0.0   1.0   0.0   
...                           ...   ...  ...   ...   ...   ...   ...   ...   
2024-05-23 00:00:00 ZNPU1    17.0  24.0  ...   0.0   0.0   0.0   0.0   0.0   
                    ZNRN5     9.0  24.0  ...   0.0   0.0   0.0   0.0   0.0   
                    ZONM8    22.0  26.0  ...   0.0   0.0   0.0  53.0   0.0   
                    ZSFO1    24.0  25.0  ...   0.0   0.0   0.0  52.0   0.0   
                    ZVLT2    26.0  17.0  ...   0.0   0.0   0.0   7.0   0.0   

                               SLV  I12   S24   SOL   SWH  
datetime            site_id                                
2024-05-16 00:00:00 086092   120.0  0.0        54.0   2.0  
                    188557   100.0  0.0        12.0   1.0  
                    220792    95.0  0.0        68.0   0.0  
                    2A1      110.0  0.0        30.0   NaN  
                    2M2      100.0  0.0        44.0   NaN  
...                            ...  ...   ...   ...   ...  
2024-05-23 00:00:00 ZNPU1     94.0  0.0   NaN  64.0   NaN  
                    ZNRN5    100.0  0.0   NaN  67.0   NaN  
                    ZONM8     69.0  0.0   NaN   5.0   NaN  
                    ZSFO1    100.0  0.0   NaN  13.0   NaN  
                    ZVLT2    120.0  0.0   NaN  66.0   NaN  

[143835 rows x 29 columns]